# UPLOAD AND TRANSFORM INTERVIEWS

In [1]:
import os
import re
import pandas as pd
from IPython.display import display
import tkinter as tk
from tkinter import filedialog
from collections import defaultdict
from tqdm.notebook import tqdm
import json
from openai import OpenAI

In [ ]:
# OpenAI client setup
import os
from openai import OpenAI

# Option 1 (recommended): read from env var
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Option 2 (manual, not recommended): paste a key here temporarily
OPENAI_API_KEY = "YOUR KEY HERE!"

# Try loading from .env if not already set
if not OPENAI_API_KEY:
    try:
        from dotenv import load_dotenv  # pip install python-dotenv
        load_dotenv()
        OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    except Exception:
        pass

if not OPENAI_API_KEY:
    raise RuntimeError(
        "OPENAI_API_KEY is not set. Set it as an env var or in a local .env file."
    )

client = OpenAI(api_key=OPENAI_API_KEY)



In [ ]:
def upload_txt_files():
    root = tk.Tk()
    root.withdraw()
    file_paths = filedialog.askopenfilenames(
        title="Select Interview Text Files",
        filetypes=[("Text files", "*.txt")]
    )
    return list(file_paths)

file_paths = upload_txt_files()
print(f"Uploaded {len(file_paths)} file(s).")

In [ ]:
def parse_transcript(path, interview_id=None):
    with open(path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    # Find the start of the transcript section
    try:
        start_idx = lines.index("Transcript")
        lines = lines[start_idx + 1:]
    except ValueError:
        print(f"'Transcript' section not found in: {path}")
        return pd.DataFrame(columns=["speaker", "text", "time", "interview_id"])

    records = []
    i = 0
    while i < len(lines) - 1:
        # Match timestamp and speaker, e.g., 0:03 | Emiliano
        match = re.match(r"^(\d{1,2}:\d{2})\s+\|\s+(.*)", lines[i])
        if match:
            timestamp = match.group(1).strip()
            speaker = match.group(2).strip()
            text = lines[i + 1].strip()
            records.append({
                "speaker": speaker,
                "text": text,
                "time": timestamp,
                "interview_id": interview_id
            })
            i += 2
        else:
            i += 1  # Skip lines that don't match the pattern

    return pd.DataFrame(records)


In [ ]:
all_dfs = []

for path in file_paths:
    interview_id = os.path.basename(path).replace(".txt", "")
    df = parse_transcript(path, interview_id=interview_id)  # pass interview_id directly
    all_dfs.append(df)

# Combine all interview transcripts into one DataFrame
all_data = pd.concat(all_dfs, ignore_index=True)
display(all_data.head())

In [ ]:
# Filter out the interviewer's responses
interviewer_names = ["YOUR NAME HERE"]
all_data_filtered = all_data[~all_data["speaker"].str.lower().isin([n.lower() for n in interviewer_names])]

display(all_data_filtered.head())

In [ ]:
# Custom save path
save_path = "YOUR PATH HERE!"
all_data_filtered.to_csv(save_path, index=False)
print(f"Saved to {save_path}")


# PROCESS INTERVIEWS AND CREATE OPEN CODES

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# Load your cleaned interview file
file_path = save_path  # This should already be defined in your notebook
df = pd.read_csv(file_path)

# Filter out short/irrelevant utterances
df["text"] = df["text"].astype(str)
df["word_count"] = df["text"].apply(lambda x: len(x.split()))
df_filtered = df[df["word_count"] >= 5].copy()

In [ ]:
# Estimate token cost before running
n_rows = len(df_filtered)
avg_tokens_per_prompt = 150  # rough estimate (input + output)
total_est_tokens = n_rows * avg_tokens_per_prompt
price_per_1k_tokens = 0.03  # GPT-4 estimate (input + output)
estimated_cost = (total_est_tokens / 1000) * price_per_1k_tokens

print(f"🧮 Estimated API cost for {n_rows} responses: ~{estimated_cost:.2f} USD ({total_est_tokens} tokens)\n")

In [ ]:
# Function to get open code and rationale
def get_code_and_reason(text):
    prompt = f"""
You are a qualitative researcher. Your task is to assign a short open code (2–4 words) to the following interview response and explain why you chose it.

Text:
\"{text}\"

Respond in JSON format:
{{
  "code": "short open code",
  "rationale": "brief explanation of why this code fits"
}}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4
        )
        content = response.choices[0].message.content
        parsed = json.loads(content)
        return parsed.get("code", ""), parsed.get("rationale", "")
    except Exception as e:
        return "ERROR", f"ERROR: {str(e)}"

In [ ]:
codes, rationales = [], []
for row in tqdm(df_filtered.itertuples(), total=len(df_filtered)):
    code, rationale = get_code_and_reason(row.text)
    codes.append(code)
    rationales.append(rationale)

df_filtered["code"] = codes
df_filtered["code_rationale"] = rationales
df_filtered.drop(columns=["word_count"], inplace=True)

In [ ]:
unique_codes = df_filtered["code"].dropna().unique()
code_list_str = ", ".join(unique_codes)

cluster_prompt = f"""
You are a qualitative analyst. Group the following open codes into 4–10 high-level themes.
For each theme, provide a brief explanation and list the codes it contains.

Respond in JSON format like this
{{
  "Theme Name 1": {{
    "rationale": "explanation of this theme",
    "codes": ["code1", "code2", ...]
  }},
  ...
}}
Codes:
{code_list_str}
"""

theme_response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": cluster_prompt}],
    temperature=0.3,
)

theme_data = theme_response.choices[0].message.content
themes_dict = json.loads(theme_data)


In [ ]:
unique_codes = df_filtered["code"].dropna().unique()
code_list_str = ", ".join(unique_codes)

cluster_prompt = f"""
You are a qualitative analyst. Group the following open codes into 4–10 high-level themes.
For each theme, provide a brief explanation and list the codes it contains.

Respond in JSON format like this
{{
  "Theme Name 1": {{
    "rationale": "explanation of this theme",
    "codes": ["code1", "code2", ...]
  }},
  ...
}}
Codes:
{code_list_str}
"""

theme_response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": cluster_prompt}],
    temperature=0.3,
)

theme_data = theme_response.choices[0].message.content
themes_dict = json.loads(theme_data)


In [ ]:
code_to_theme = {}
code_to_theme_rationale = {}

for theme, info in themes_dict.items():
    rationale = info.get("rationale", "")
    for c in info.get("codes", []):
        code_to_theme[c.lower()] = theme
        code_to_theme_rationale[c.lower()] = rationale

df_filtered["theme"] = df_filtered["code"].apply(lambda x: code_to_theme.get(x.lower(), "Unclustered"))
df_filtered["theme_rationale"] = df_filtered["code"].apply(lambda x: code_to_theme_rationale.get(x.lower(), ""))

In [ ]:
# 🧭 Print Themes, Rationales, and Assigned Codes
print("\n🧭 Themes & Rationales and Their Codes:")
assigned_codes = set()

for theme, info in themes_dict.items():
    codes = info.get("codes", [])
    rationale = info.get("rationale", "[No rationale provided]")

    if not codes:
        print(f"\n⚠️ Skipping theme '{theme}' (no codes).")
        continue

    print(f"\n🔹 {theme} ({len(codes)} codes):\n  Rationale: {rationale}")
    for code in sorted(codes):
        print(f"   - {code}")
        assigned_codes.add(code.lower())  # normalize for matching

# 🔍 Identify unassigned (unclustered) codes
all_codes = set(df_filtered["code"].dropna().str.lower().unique())
unclustered_codes = all_codes - assigned_codes

if unclustered_codes:
    print(f"\n❌ Unclustered Codes ({len(unclustered_codes)}):")
    for code in sorted(unclustered_codes):
        print(f"   - {code}")
else:
    print("\n✅ All codes were clustered.")

In [ ]:
output_path = "coded_transcripts_with_themes_and_rationale.csv"
df_filtered.to_csv(output_path, index=False)
print(f"\n💾 Saved to {output_path}")

# CREATE CLOSED CODES AND RE-PROCESS

In [ ]:
closed_codebook = {
        "Experience with data connectors": "Refers to the interviewee's experience with data connectors, including their familiarity with different types of connectors and their ability to use them effectively.",
        "Experience with tasks": "Refers to the interviewee's experience using tasks, including their familiarity with different types of tasks and their ability to use them effectively.",
        "Pain-points or surprises during setup": "Refers to the interviewee's setting up data connectors, including any issues or challenges they have encountered during the setup process.",
        "Expectations vs reality": "Refers to the interviewee's expectations and reality of using data connectors, including any differences between their expectations and the actual experience.",
        "Data connectors and/or Tasks success metrics": "Refers to the interviewee's plan to measure the success of the usage of data connectors or tasks after the implementation. This includes how they decided to measure the success of the usage of data connectors or tasks after the implementation.",
        "What have data connectors and/or tasks changed for the team or company": "Refers to the interviewee's observations on how the usage of data connectors or tasks has changed the team or company, including any positive or negative impacts.",
        "Business case and ROI": "Refers to how the interviewee has created a business case to received resources from the company (for example, in terms of time or engineering or development resources) in order to implement data connectors or tasks, as well as the expectation of the ROI this implementation will bring.",
        "Data connectors and Task reporting": "Refers to how the interviewee monitors the usage of data connectors or tasks, including any reporting or dashboards they have created to track the usage of data connectors or tasks, or lack thereof.",
        "Experienced task problems or issues": "Refers to any issues the interviewee or the extended team has experienced setting up a task. This includes any issues with the setup interface, prompting, testing, etc.",
        "Prompting experience": "Prompting is an important part of using data connectors or tasks, since the AI agent must know when to trigger them and what procedure to follow. This refers to the experience and challenges the interviewee or the extended team have faced creating prompts, as well as their perceived ability and skills in creating prompts.",
        "Trust in data connectors and/or Tasks":"Refers to the interviewee's trust in the data connectors or tasks, including any concerns or doubts they have about the reliability or effectiveness of the data connectors or tasks, and in which scenarios or use cases they would not use them.",
        "KPIs": "Refers to the interviewee's organisational KPIs when it comes to data connectors or tasks. Mainly, what the company is trying to achieve with data connectors or tasks in the long term.",
        "Testing data connectors and/or Tasks": "Refers to the interviewee's experience testing data connectors or tasks, including any issues or challenges they have encountered during the testing process (for example, understanding why a task doesn't trigger, at what point data connectors are called and visibility of the logs).",
}

In [ ]:
avg_tokens_per_prompt_closed = 200  # slightly longer than open coding
est_tokens_closed = len(df_filtered) * avg_tokens_per_prompt_closed
est_cost_closed = (est_tokens_closed / 1000) * 0.03  # gpt-4 estimated pricing

print(f"🧮 Estimated cost for closed coding: ~{est_cost_closed:.2f} USD ({est_tokens_closed} tokens)")

In [ ]:
def assign_closed_code(text, codebook):
    code_options = "\n".join([f'"{k}": {v}' for k, v in codebook.items()])
    prompt = f"""
You are a qualitative analyst specializing in customer interview research. Your job is to assign **one best-fit code** from the list below to this interview response.

CONTEXT: Some context about the topic of the interviews.

CODING INSTRUCTIONS:
- Choose the code that best captures the PRIMARY theme of the response
- If the response doesn't clearly fit any code, select "None"
- Focus on the main insight or experience being shared
- Consider both explicit statements and implied meanings

Codes:
{code_options}

Response:
\"{text}\"

Respond in JSON format:
{{
  "code": "selected code or None",
  "rationale": "brief reason why this code fits"
}}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        content = response.choices[0].message.content
        parsed = json.loads(content)
        return parsed.get("code", "None"), parsed.get("rationale", "")
    except Exception as e:
        return "ERROR", f"ERROR: {str(e)}"

In [ ]:
code_closed_list = []
rationale_closed_list = []

for row in tqdm(df_filtered.itertuples(), total=len(df_filtered)):
    code, rationale = assign_closed_code(row.text, closed_codebook)
    code_closed_list.append(code)
    rationale_closed_list.append(rationale)

df_filtered["code_closed"] = code_closed_list
df_filtered["code_closed_rationale"] = rationale_closed_list

In [ ]:
# Get unique closed codes (excluding None/error)
valid_codes = df_filtered["code_closed"].dropna().unique()
valid_codes = [c for c in valid_codes if c.lower() not in ["none", "error"]]
codes_str = ", ".join(valid_codes)

theme_prompt = f"""
You are a qualitative researcher. Cluster the following codes into 4–8 higher-level themes.

For each theme, provide:
- A name
- A rationale
- The codes that belong in it

Respond in JSON like this:
{{
  "Theme Name": {{
    "rationale": "why these codes belong together",
    "codes": ["code1", "code2", ...]
  }}
}}

Codes:
{codes_str}
"""

theme_response_closed = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": theme_prompt}],
    temperature=0.3,
)

theme_data_closed = theme_response_closed.choices[0].message.content
themes_dict_closed = json.loads(theme_data_closed)

In [ ]:
# Map each closed code to its clustered theme + rationale
code_to_theme_closed = {}
code_to_theme_rationale_closed = {}

for theme, info in themes_dict_closed.items():
    rationale = info.get("rationale", "")
    for c in info.get("codes", []):
        code_to_theme_closed[c.lower()] = theme
        code_to_theme_rationale_closed[c.lower()] = rationale

df_filtered["theme_closed"] = df_filtered["code_closed"].apply(
    lambda x: code_to_theme_closed.get(x.lower(), "Unclustered") if isinstance(x, str) else "Unclustered"
)

df_filtered["theme_closed_rationale"] = df_filtered["code_closed"].apply(
    lambda x: code_to_theme_rationale_closed.get(x.lower(), "") if isinstance(x, str) else ""
)

In [ ]:
print(f"📊 Rows processed: {len(df_filtered)}")
print(f"🔖 Unique closed codes (excluding 'None'): {len(valid_codes)}")
print(f"🧠 Themes from closed codes: {len(themes_dict_closed)}")

print("\n🧭 Closed Code Themes:")
for theme, info in themes_dict_closed.items():
    print(f"\n🔹 {theme} ({len(info['codes'])} codes):\n  Rationale: {info['rationale']}")
    for code in sorted(info["codes"]):
        print(f"   - {code}")

# Show unclustered closed codes
all_codes_closed = set([c.lower() for c in valid_codes])
assigned_closed = set(code_to_theme_closed.keys())
unclustered_closed = all_codes_closed - assigned_closed

if unclustered_closed:
    print(f"\n❌ Unclustered closed codes ({len(unclustered_closed)}):")
    for c in sorted(unclustered_closed):
        print(f"   - {c}")

In [ ]:
output_path_closed = "coded_transcripts_with_closed_coding.csv"
df_filtered.to_csv(output_path_closed, index=False)
print(f"\n💾 Saved to {output_path_closed}")

# CREATE REPORT ON CLOSED CODES

In [ ]:
from random import sample

def generate_report(df):
    # Filter out unclustered or NaN themes
    themes = df["theme_closed"].dropna().unique()
    themes = [t for t in themes if t.lower() != "unclustered"]

    theme_sections = []

    for theme in themes:
        rationale_series = df[df["theme_closed"] == theme]["theme_closed_rationale"].dropna()
        if rationale_series.empty:
            continue
        rationale = rationale_series.iloc[0]

        # Use verbatim quotes (text) from the CSV
        quotes_df = df[df["theme_closed"] == theme][["text", "interview_id"]].dropna()
        if quotes_df.empty:
            continue

        # Select 1–3 quotes randomly
        quotes_sample = quotes_df.sample(n=min(3, len(quotes_df)), random_state=42)

        quotes_text = ""
        for _, row in quotes_sample.iterrows():
            text = row["text"].strip()  # leave as-is for CSV matching
            interview_id = row["interview_id"]
            quotes_text += f'"{text}"\nInterview ID: {interview_id}\n\n'

        section = f"""### {theme}

**Theme Description:** {rationale}

**Supporting Quotes:**
{quotes_text}"""
        theme_sections.append(section)

    if not theme_sections:
        return "⚠️ No valid themes with quotes and rationales were found."

    theme_content = "\n".join(theme_sections)

    report_prompt = f"""
You are a research analyst. Write a clear, professional report based on the following themes and **verbatim participant quotes** from user interviews.

### Instructions:
- Begin with an **Introduction** summarizing what this report covers.
- For each **Theme**, use the exact quotes provided without paraphrasing.
- Include the quotes in bullet form or as blockquotes, preserving their original wording.
- For each quote, also include the **Interview ID** below it.
- For each theme, also include a summary of the theme findings.
- At the end, write a **Summary** that highlights overall takeaways.
- Finally, offer **Recommendations** based on the main insights from all themes.

Here are the verbatim themes and quotes:
{theme_content}
"""
    return report_prompt


In [ ]:
# Generate structured prompt
report_prompt = generate_report(df_filtered)

# Call GPT-4 to generate the report
report_response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": report_prompt}],
    temperature=0.5,
)

report_text = report_response.choices[0].message.content

# Print the result
print(report_text)

In [ ]:
with open("final_interview_report.txt", "w", encoding="utf-8") as f:
    f.write(report_text)

print("📄 Report saved to final_interview_report.txt")